In [11]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_blobs

In [27]:
class MyKMeans:
    def __init__(self, n_clusters=3, max_iter=10, n_init=3, random_state=42):
        self.n_clusters = n_clusters
        self.max_iter = max_iter
        self.n_init = n_init
        self.random_state = random_state
    
    def __str__(self):
        # Формируем строку с параметрами экземпляра
        params = vars(self)
        params_str = ', '.join(f"{key}={value}" for key, value in params.items())
        return f"MyKMeans class: {params_str}"
    
    def fit(self, X):
        np.random.seed(self.random_state)
        X = X.to_numpy()  # Преобразуем датафрейм в массив numpy
        n_samples, n_features = X.shape
        best_inertia = float('inf')
        best_centers = None

        for _ in range(self.n_init):
            # Инициализация случайных центров кластеров
            centers = np.array([
                [np.random.uniform(np.min(X[:, j]), np.max(X[:, j])) for j in range(n_features)]
                for _ in range(self.n_clusters)
            ])
            
            for iteration in range(self.max_iter):
                # Шаг 1: Распределение точек по кластерам
                labels = np.argmin(
                    np.linalg.norm(X[:, np.newaxis] - centers, axis=2), axis=1
                )
                
                # Шаг 2: Пересчет центров кластеров
                new_centers = np.array([
                    X[labels == k].mean(axis=0) if len(X[labels == k]) > 0 else centers[k]
                    for k in range(self.n_clusters)
                ])
                
                # Если центры не изменились, завершить итерации
                if np.allclose(centers, new_centers):
                    break
                centers = new_centers
            
            # Вычисление WCSS (внутрикластерной суммы квадратов)
            inertia = sum(
                np.sum((X[labels == k] - centers[k])**2)
                for k in range(self.n_clusters)
            )
            
            # Сохранение лучшей модели
            if inertia < best_inertia:
                best_inertia = inertia
                best_centers = centers

        self.cluster_centers_ = best_centers.tolist()
        self.inertia_ = best_inertia
        
    def predict(self, X):
        # Проверка, что модель обучена
        if self.cluster_centers_ is None:
            raise ValueError("Model has not been trained. Call fit().")

        # Преобразование входных данных в массив numpy
        X = X.to_numpy()

        # Вычисление расстояний от точек до центров кластеров
        distances = np.linalg.norm(X[:, np.newaxis] - self.cluster_centers_, axis=2)

        # Для каждой точки находим индекс ближайшего центра
        labels = np.argmin(distances, axis=1)

        return labels.tolist()

In [26]:
#Тест предсказания
# Генерация данных
X, y = make_blobs(n_samples=10, centers=10, n_features=2, random_state=42)
data = pd.DataFrame(X, columns=['feature1', 'feature2'])

# Инициализация и обучение модели
params = {"n_clusters": 10, "max_iter": 10, "n_init": 3}
kmeans = MyKMeans(**params)
kmeans.fit(data)

# Предсказание кластеров
predictions = kmeans.predict(data)

# Вывод результатов
print(predictions)

[2, 8, 0, 8, 9, 6, 4, 1, 6, 9]


In [21]:
#Тест обучения
# Генерация тестовых данных с 2 фичами и 2 кластерами
X, y = make_blobs(n_samples=20, centers=2, n_features=2)
data = pd.DataFrame(X, columns=['feature1', 'feature2'])

# Инициализация и обучение модели
kmeans = MyKMeans(n_clusters=2, max_iter=50, n_init=5)
kmeans.fit(data)

# Результаты
print("Centroids:", kmeans.cluster_centers_)
print("WCSS (Inertia):", kmeans.inertia_)


Centroids: [[-4.237358706734224, -2.645892201438761], [2.0695759547461265, -7.574591078045801]]
WCSS (Inertia): 30.35551333817684


In [28]:
#Тест класса
model1 = MyKMeans()
model2 = MyKMeans(n_clusters=5, n_init=4)
model3 = MyKMeans(max_iter=20)

# Проверка
print(model1)
print(model2)
print(model3)

MyKMeans class: n_clusters=3, max_iter=10, n_init=3, random_state=42
MyKMeans class: n_clusters=5, max_iter=10, n_init=4, random_state=42
MyKMeans class: n_clusters=3, max_iter=20, n_init=3, random_state=42
